In [9]:
from MAUtil import *
from MAInit import *
from MACalc import *
from MAPost import *

In [10]:
### get file names ###
env = 'local'
files = os.listdir('./database/')
filenames = []

words = ['E_no','u4']
nwords = ['__', 'all']

for i in range(len(files)):
    if all((s in files[i]) for s in words):
        if all((s not in files[i]) for s in nwords):
#     if 'E_no' in files[i]:
#         if '__' not in files[i]:
            filenames.append(files[i])
            print(i, files[i])

3168 Pd_100_u4_RPBE_no001_CO_n12_d3.traj
3182 Pd_100_u4_RPBE_no002_CO_n8_d3.traj
3186 Pd_100_u4_RPBE_no003_CO_n10_d3.traj
3194 Pd_100_u4_RPBE_no003_CO_n8_d3.traj
3816 Pt_111_u4_RPBE_no001_CO_n10_d3.traj
3820 Pt_111_u4_RPBE_no001_CO_n12_d3.traj
3826 Pt_111_u4_RPBE_no001_CO_n8_d3.traj
3830 Pt_111_u4_RPBE_no002_CO_n10_d3.traj
3834 Pt_111_u4_RPBE_no002_CO_n12_d3.traj
3840 Pt_111_u4_RPBE_no002_CO_n8_d3.traj
3848 Pt_111_u4_RPBE_no003_CO_n6_d3.traj


In [11]:
# ### DELETE ###
# for file in filenames:
#     try:
#         A = make_database(file, 'adsE_method2_new')
#         A.delete_from_database()
#     except:
#         print(file, 'Some error occured.')

-----------------------------------------------------------
Pd_100_u4_RPBE_no001_CO_n12_d3.traj
Pd_100_u4_RPBE_no001_CO_n12_d3.traj deleted
-----------------------------------------------------------
Pd_100_u4_RPBE_no002_CO_n8_d3.traj
Pd_100_u4_RPBE_no002_CO_n8_d3.traj deleted
-----------------------------------------------------------
Pd_100_u4_RPBE_no003_CO_n10_d3.traj
Pd_100_u4_RPBE_no003_CO_n10_d3.traj deleted
-----------------------------------------------------------
Pd_100_u4_RPBE_no003_CO_n8_d3.traj
Pd_100_u4_RPBE_no003_CO_n8_d3.traj deleted
-----------------------------------------------------------
Pt_111_u4_RPBE_no001_CO_n10_d3.traj
Pt_111_u4_RPBE_no001_CO_n10_d3.traj deleted
-----------------------------------------------------------
Pt_111_u4_RPBE_no001_CO_n12_d3.traj
Pt_111_u4_RPBE_no001_CO_n12_d3.traj deleted
-----------------------------------------------------------
Pt_111_u4_RPBE_no001_CO_n8_d3.traj
Pt_111_u4_RPBE_no001_CO_n8_d3.traj deleted
--------------------------

In [8]:
### ADD and UPDATE ENERGY ###
for file in filenames:
#     try:
    A = make_database(file, 'adsE_method2_new')
    A.add_to_database()
    A.update_Energy()
    A.update_adsorbates_correlation(expression=2, force_update=False)
#     except:
#         print(file, 'Some error occured.')

-----------------------------------------------------------
Pd_100_u4_RPBE_no001_CO_n12_d3.traj
Not in database.
One post: 5f0a3d03bdb65c1c5cb188a1

E_each_ads and E_residue/suratom updated.
Adsorbate correlation updated.
-----------------------------------------------------------
Pd_100_u4_RPBE_no002_CO_n8_d3.traj
Not in database.
One post: 5f0a3d5abdb65c1c5cb188a3

E_each_ads and E_residue/suratom updated.
Adsorbate correlation updated.
-----------------------------------------------------------
Pd_100_u4_RPBE_no003_CO_n10_d3.traj
Not in database.
One post: 5f0a3dacbdb65c1c5cb188a5

E_each_ads and E_residue/suratom updated.
Adsorbate correlation updated.
-----------------------------------------------------------
Pd_100_u4_RPBE_no003_CO_n8_d3.traj
Not in database.
One post: 5f0a3e01bdb65c1c5cb188a7

E_each_ads and E_residue/suratom updated.
Adsorbate correlation updated.
-----------------------------------------------------------
Pt_111_u4_RPBE_no001_CO_n10_d3.traj
Not in database.
O

In [8]:
file = 'Zn_100_u2_RPBE_no899_CO_n3_d3.traj'
A = make_database(file, 'adsE_method2')
A.check_database()

-----------------------------------------------------------
Zn_100_u2_RPBE_no899_CO_n3_d3.traj


{'_id': ObjectId('5e8b856122a772c44608a0f2'),
 'name': 'Zn_100_u2_RPBE_no899_CO_n3_d3.traj',
 'isvalid': 'no',
 'ispredictable': 'yes',
 'element': 'Zn',
 'face': '100',
 'unitlength': 2,
 'xc': 'RPBE',
 'adsorbate': 'CO',
 'numberofads': 3,
 'coverage': 0.75,
 'surfatomnum': 4,
 'E': -51.03105548,
 'bareE': -7.45475945,
 'E_ads': -14.48543631,
 'totaladsE': -0.1199870999999959,
 'aveadsE/suratom': -0.029996774999998976,
 'aveadsE/ads': -0.039995699999998635,
 'E_int_space': -0.004996649999995384,
 'sumE_each_ads': -0.1509804999999922,
 'E_residue/suratom': 0.008997512499997917,
 'area': 56.86330592504613,
 'density': 0.052758100346019694,
 'igroups': [3, 3, 1],
 'rgroups': [3, 1, 1],
 'converged': 'yes',
 'is_adsorbed': 'yes',
 'kept_sites': 'no',
 'E_not_exceeded': 'yes',
 'minimum_distance': 3,
 'ads_dist2': 0.0,
 'ads_dist3': 4.0}

In [18]:
query('Zn_001_u2_RPBE_no003_CO_n1_d8.traj','local')

Atoms(symbols='Zn16CO', pbc=True, cell=[[5.323107383026382, 0.0, 3e-16], [-2.661553691513193, 4.609946220773349, 3e-16], [0.0, 0.0, 37.388231425177516]], constraint=FixAtoms(indices=[0, 1, 4, 5, 8, 9, 12, 13]), calculator=SinglePointCalculator(...))